Extract Example
=======================

In [ ]:
%matplotlib inline

import numpy as np
import tables as tb
import matplotlib.pyplot as plt

from astropy.table import Table, Column, join
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import fits

from hetdex_api.extract import Extract
from hetdex_api.config import HDRconfig
from hetdex_tools.get_spec import get_spectra
from hetdex_api.shot import get_fibers_table, open_shot_file
from hetdex_api.detections import Detections

In [ ]:
%matplotlib inline

In [ ]:

plt.style.use('default')
plt.rcParams['axes.linewidth'] = 2
plt.rcParams.update({'font.size': 14})

plt.rcParams['lines.linewidth'] = 2

plt.rcParams['font.family'] = 'serif'
plt.rcParams['mathtext.fontset'] = 'dejavuserif'
plt.rcParams['xtick.direction']= 'in'
plt.rcParams['ytick.direction']= 'in'
plt.rcParams['xtick.labelsize']= 12.0
plt.rcParams['ytick.labelsize']= 12.0

In [ ]:
# Example Extraction for a White Dwarf in HDR4 continuum catalog. Using the Detections API just to get coordinates, shotid and fwhm. It is not necessary

C_hdr4 = Detections('hdr4', catalog_type='continuum')

detectid = 4090341661
coord = C_hdr4.get_coord(detectid)
survey_info = C_hdr4.get_survey_info(detectid)[0]
det_info = C_hdr4.get_detection_info(detectid)[0]

shotid = det_info['shotid']
fwhm = survey_info['fwhm_virus']

# get H5 spectrum just for comparison here
spec = C_hdr4.get_spectrum(detectid, deredden=False)

In [ ]:
spec_tab = get_spectra( coords=coord, shotid=shotid, survey='hdr4', loglevel='WARNING')

In [ ]:
E = Extract()
E.load_shot(shotid)
radius=3.5
ffsky=False

# can use Extracts get_fiberinfo_for_coord or get_fibers_table from hetdex_api.shot. These are the
# same behind code bits. get_fiberinfo_for_coord starts with get_fibers_table then gets
# x,y coordinates on IFU and masking data
use_extract = False

if True:

    info_result = E.get_fiberinfo_for_coord(coord, radius=radius, verbose=True, ffsky=ffsky)
    ifux, ifuy, xc, yc, ra, dec, data, error, mask = info_result

else: 
    fib_table = get_fibers_table(
        shotid, 
        coord,
        survey=survey,
        radius=radius,
        verbose=True,
        astropy=False,
        F=E.fibers, # pass through Fibers class object to keep H5 open, will open if set to None
        fiber_flux_offset=None
    )
    
    fiber_lower_limit=3 # this is built into get_spectra. Won't extract on less than 3 fibers
    
    if np.size(fib_table) < fiber_lower_limit:
        print('Not enough fibers')

    ifux = fib_table["ifux"]
    ifuy = fib_table["ifuy"]
    ra = fib_table["ra"]
    dec = fib_table["dec"] 
    
    if ffsky:
        if self.survey == 'hdr2.1':
            spec = fib_table["spec_fullsky_sub"]
        else:
            spec = fib_table["calfib_ffsky"]
    else:
        spec = fib_table["calfib"]
        spece = fib_table["calfibe"]
        ftf = fib_table["fiber_to_fiber"]

        mask = fib_table["calfibe"]
        mask = (mask > 1e-8) * (np.median(ftf, axis=1) > 0.5)[:, np.newaxis]

        ifux[:] = ifux + E.dither_pattern[expn - 1, 0]
        ifuy[:] = ifuy + E.dither_pattern[expn - 1, 1]

        xc, yc = E.convert_radec_to_ifux_ifuy(
            ifux, ifuy, ra, dec, coord.ra.deg, coord.dec.deg
        )

    
moffat = E.moffat_psf(fwhm, 10.5, 0.25)
weights = E.build_weights(xc, yc, ifux, ifuy, moffat)

# See Greg Zeimann's Remedy code
norm = np.sum(weights, axis=0)

weights = weights/norm

result = E.get_spectrum(data, error, mask, weights, remove_low_weights = False)

spectrum, spectrum_error = [res for res in result]

plt.figure(figsize=(20,4))
plt.subplot(131)
plt.plot(E.wave, spectrum/norm, lw=2, label='weight normalization')
plt.plot(E.wave, spec_tab['spec'][0], lw=2, label='get_spectra', linestyle='dashed')
plt.plot(E.wave, spec['spec1d'], label='hdr4/H5')
plt.legend()

plt.subplot(132)
plt.plot(E.wave, spectrum_error/norm, label='extract error')
plt.plot(E.wave, spec_tab['spec_err'][0], label='get_spectra error')
plt.plot(E.wave, spec['spec1d_err'], label='hdr4/h5 error')

plt.legend()

plt.subplot(133)
plt.plot(E.wave, (spectrum/norm) / (spec['spec1d']), label='extract/h5')
plt.plot(E.wave, (spectrum/norm) / (spec_tab['spec'][0]), label='extract/get_spectra')
plt.legend()

In [ ]:
# when done you should close Extract and Detection classes

E.close()
C_hdr4.close()